In [ ]:
!pip install -q torchinfo mlflow optuna thop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.5 MB/s eta 0:00:00


In [ ]:
import os
import math
import random
from collections import defaultdict, Counter
from typing import List, Optional

import h5py
import numpy as np
import pandas as pd
import pywt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score
import mlflow
import mlflow.pytorch
import optuna
from optuna.exceptions import TrialPruned
from dotenv import load_dotenv
import time
from thop import profile, clever_format

load_dotenv()

True

In [ ]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/oladipoeyiara@gmail.com/han_sph_full_rr_interval_after_tuning")


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/1222522124100997', creation_time=1768344478752, experiment_id='1222522124100997', last_update_time=1768424820958, lifecycle_stage='active', name='/Users/oladipoeyiara@gmail.com/han_sph_full_rr_interval_after_tuning', tags={'mlflow.experiment.sourceName': '/Users/oladipoeyiara@gmail.com/han_sph_full_rr_interval_after_tuning',
 'mlflow.experimentKind': 'custom_model_development',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'oladipoeyiara@gmail.com',
 'mlflow.ownerId': '1108839756692281'}>

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)

    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# Define the path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/records.zip'

# Define the extraction path
extract_path = '/content'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Check if the zip file exists
if os.path.exists(zip_file_path):
    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Successfully extracted {zip_file_path} to {extract_path}")
else:
    print(f"Error: {zip_file_path} not found.")

Mounted at /content/drive
Successfully extracted /content/drive/MyDrive/records.zip to /content


In [ ]:
# -------------------------
# Config
# -------------------------
PATH = "records"
SAMPLING_RATE = 500
BATCH_SIZE = 32
DROP_LAST = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# -------------------------
# utilities: filtering and peak detection
# -------------------------
def denoise(data):
    # wavelet transform
    coeffs = pywt.wavedec(data=data, wavelet='db5', level=9)
    cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs

    # Threshold denoising
    threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
    cD1.fill(0)
    cD2.fill(0)
    for i in range(1, len(coeffs) - 2):
        coeffs[i] = pywt.threshold(coeffs[i], threshold)

    # Inverse wavelet transform to obtain the denoised signal
    rdata = pywt.waverec(coeffs=coeffs, wavelet='db5')
    return rdata


def pan_tompkins_detector(ecg_signal, fs):
    lowcut, highcut = 5.0, 15.0
    nyquist = 0.5 * fs
    low, high = lowcut / nyquist, highcut / nyquist
    b, a = butter(1, [low, high], btype='band')
    filtered_ecg = filtfilt(b, a, ecg_signal)
    diff_ecg = np.diff(filtered_ecg)
    squared_ecg = diff_ecg ** 2
    window_size = int(0.150 * fs)
    mwa_ecg = np.convolve(squared_ecg, np.ones(window_size) / window_size, mode='same')
    peaks, _ = find_peaks(mwa_ecg, distance=int(0.6 * fs))
    return peaks


def multi_lead_fusion(detected_peaks, fs, fusion_window=0.1, min_leads=None):
    n_leads = len(detected_peaks)
    if min_leads is None:
        min_leads = int(np.ceil(n_leads / 2))

    # Collect all peaks with their lead information
    all_peaks = [(p, lead) for lead, peaks in enumerate(detected_peaks) for p in peaks]
    all_peaks.sort(key=lambda x: x[0])

    fused_peaks = []
    i = 0

    while i < len(all_peaks):
        # Start a new cluster
        cluster = [all_peaks[i]]
        i += 1

        # Add nearby peaks to the cluster
        while i < len(all_peaks) and all_peaks[i][0] - cluster[-1][0] <= fusion_window * fs:
            cluster.append(all_peaks[i])
            i += 1

        # Check if cluster has peaks from enough leads
        unique_leads = {lead for _, lead in cluster}
        if len(unique_leads) >= min_leads:
            # Use median position as the fused peak
            fused_peak = int(np.median([p for (p, _) in cluster]))
            fused_peaks.append(fused_peak)

    return np.array(sorted(fused_peaks))


def detect_r_peaks(ecg_signals, fs):
    detected_peaks = []
    for lead in ecg_signals:
        peaks = pan_tompkins_detector(lead, fs)
        detected_peaks.append(peaks)

    fused_r_peaks = multi_lead_fusion(detected_peaks, fs, fusion_window=0.1, min_leads=6)
    return fused_r_peaks


def extract_segments_around_peaks(signal, r_peaks, pre_samples, post_samples):
    segments = []

    for peak in r_peaks:
        start = max(0, peak - pre_samples)
        end = min(len(signal), peak + post_samples)

        # Only include segments with the correct length
        if end - start == pre_samples + post_samples:
            segment = signal[start:end]
            segments.append(segment)

    return segments

def extract_rr_beats_multi_lead(ecg_signals, fs, denoise_fn=None,
                                min_rr_ms=300, max_rr_ms=1500, min_beats=2):
    """
    Returns a list of beats, each beat is an array with shape (T_i, C) where C = n_leads.
    """
    ecg = np.array(ecg_signals)
    ecg = np.nan_to_num(ecg, nan=0.0, posinf=0.0, neginf=0.0)                 # (n_leads, n_samples)
    eps = 1e-8
    means = np.mean(ecg, axis=1, keepdims=True)
    stds = np.std(ecg, axis=1, keepdims=True)

    # Prevent division by zero or exploding values for flat-line signals
    # If std is close to 0, set it to 1 (result will be 0/1 = 0, which is correct for flat line)
    stds = np.where(stds < 1e-7, 1.0, stds)

    ecg = (ecg - means) / stds

    n_leads, n_samples = ecg.shape

    if denoise_fn is not None:
        ecg = np.array([denoise_fn(lead) for lead in ecg])

    # Detect fused R-peaks once across leads
    r_peaks = detect_r_peaks(ecg, fs)            # uses your multi-lead fusion
    if len(r_peaks) < min_beats:
        return None  # not enough beats

    # RR in samples/ms
    rr_samples = np.diff(r_peaks)
    rr_ms = (rr_samples / fs) * 1000.0
    valid = (rr_ms >= min_rr_ms) & (rr_ms <= max_rr_ms)
    if valid.sum() == 0:
        return None

    beats = []
    for i in range(len(r_peaks) - 1):
        if not valid[i]:
            continue
        start = r_peaks[i]
        end   = r_peaks[i+1]
        seg = ecg[:, start:end].T                # (T_i, C)

        if np.isnan(seg).any():
            continue
        if seg.shape[0] == 0:
            continue

        if seg.shape[0] > 0:
            beats.append(seg.astype(np.float32))

    if len(beats) == 0:
        return None
    return beats  # list of variable-length (T_i, C)

In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
from collections import Counter
from joblib import Parallel, delayed
import multiprocessing as mp

# -------------------------
metadata = pd.read_csv("metadata.csv")
codes = pd.read_csv("code.csv")

code_mappings = dict(zip(codes['Code'].astype(str), codes['Category']))

metadata = metadata[~metadata['AHA_Code'].str.contains(";")]
metadata['AHA_Code'] = metadata['AHA_Code'].str.split("+").str[0]
metadata['AHA_Code_Mapped'] = metadata['AHA_Code'].map(code_mappings)

unique_labels = sorted(metadata['AHA_Code_Mapped'].dropna().unique())
label2idx = {lbl: i for i, lbl in enumerate(unique_labels)}
num_classes = len(unique_labels)

print("num_classes:", num_classes, "labels:", unique_labels)

def process_row(row):
    ecg_data_path = os.path.join(PATH, row.ECG_ID + ".h5")
    if not os.path.exists(ecg_data_path):
        return None
    try:
        with h5py.File(ecg_data_path, 'r') as f:
            ecg = np.array(f['ecg'])  # shape (n_leads, n_samples)
    except Exception as e:
        print("error reading", ecg_data_path, e)
        return None

    beats = extract_rr_beats_multi_lead(ecg, fs=SAMPLING_RATE, denoise_fn=denoise)
    if beats is None:
        return None

    label = row.AHA_Code_Mapped
    if pd.isna(label):
        return None
    if label not in label2idx:
        return None

    return ((beats, label2idx[label]))

# Use all physical cores by default
n_jobs = max(1, mp.cpu_count() - 1)

results = Parallel(
    n_jobs=n_jobs,
    backend="loky",
    verbose=10
)(
    delayed(process_row)(row)
    for row in metadata.itertuples(index=False)
)

data_list = [r for r in results if r is not None]


num_classes: 11 labels: ['A', 'C', 'D', 'E', 'F', 'H', 'I', 'J', 'K', 'L', 'M']


[Parallel(n_jobs=11)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=11)]: Done   3 tasks      | elapsed:    1.7s
[Parallel(n_jobs=11)]: Done  10 tasks      | elapsed:    1.8s
[Parallel(n_jobs=11)]: Done  19 tasks      | elapsed:    2.0s
[Parallel(n_jobs=11)]: Done  28 tasks      | elapsed:    2.1s
[Parallel(n_jobs=11)]: Done  39 tasks      | elapsed:    2.2s
[Parallel(n_jobs=11)]: Done  50 tasks      | elapsed:    2.3s
[Parallel(n_jobs=11)]: Done  63 tasks      | elapsed:    2.4s
[Parallel(n_jobs=11)]: Done  76 tasks      | elapsed:    2.6s
[Parallel(n_jobs=11)]: Done  91 tasks      | elapsed:    2.7s
[Parallel(n_jobs=11)]: Done 106 tasks      | elapsed:    2.9s
[Parallel(n_jobs=11)]: Done 123 tasks      | elapsed:    3.0s
[Parallel(n_jobs=11)]: Batch computation too fast (0.19748764738174937s.) Setting batch_size=2.
[Parallel(n_jobs=11)]: Done 140 tasks      | elapsed:    3.2s
[Parallel(n_jobs=11)]: Batch computation too fast (0.17038679122924805s.) Setting b

In [ ]:
# ---------- Length-bucketing Batch Sampler ----------
class LengthBucketBatchSampler(Sampler):
    def __init__(self,
                 lengths: List[int],
                 batch_size: int,
                 bin_size: Optional[int] = None,
                 shuffle: bool = True,
                 drop_last: bool = False):
        self.lengths = list(lengths)
        self.batch_size = int(batch_size)
        self.bin_size = bin_size
        self.shuffle = shuffle
        self.drop_last = drop_last

        # Build mapping length_key -> list of indices
        self._buckets = defaultdict(list)
        for idx, L in enumerate(self.lengths):
            key = self._length_key(L)
            self._buckets[key].append(idx)

        # Convert to normal dict for iteration; keep keys list stable
        self.bucket_keys = list(self._buckets.keys())

    def _length_key(self, length: int) -> int:
        if self.bin_size is None or self.bin_size <= 0:
            return int(length)   # exact-length bucket
        else:
            return (length // self.bin_size) * self.bin_size

    def __iter__(self):
      # For each epoch, build batches from buckets.
      batches = []
      for key in self.bucket_keys:
          idxs = list(self._buckets[key])
          if len(idxs) < self.batch_size:
              # skip this bucket entirely
              continue
          if self.shuffle:
              random.shuffle(idxs)
          # chunk into batches
          for i in range(0, len(idxs), self.batch_size):
              batch = idxs[i:i + self.batch_size]
              if len(batch) < self.batch_size and self.drop_last:
                  continue
              batches.append(batch)

      if self.shuffle:
          random.shuffle(batches)

      for batch in batches:
          yield batch

    def __len__(self):
        total = 0
        for key in self.bucket_keys:
            n = len(self._buckets[key])
            if self.drop_last:
                total += n // self.batch_size
            else:
                total += math.ceil(n / self.batch_size)
        return total

In [ ]:
# -------------------------
# Dataset + Sampler + DataLoader
# # -------------------------

class ECGSegmentDatasetVarLen(Dataset):
    def __init__(self, data_list):
        self.data = data_list
        # number of beats per record for length-bucketing by S
        self.num_beats = [len(x[0]) for x in data_list]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        beats_list, label = self.data[idx]
        # convert each beat to torch tensor
        beats_tensors = [torch.from_numpy(b) if isinstance(b, np.ndarray) else torch.tensor(b)
                         for b in beats_list]  # [(T_i, C), ...]
        return {"beats": beats_tensors, "label": int(label), "num_beats": len(beats_tensors)}


def collate_by_num_beats(batch):
    s_vals = [item["num_beats"] for item in batch]
    if not all(s == s_vals[0] for s in s_vals):
        raise ValueError("collate_by_num_beats received mixed num_beats in a batch")
    signals = torch.stack([item["signal"] for item in batch], dim=0)  # (B, S, T, C)
    labels = torch.tensor([item["label"] for item in batch], dtype=torch.long)
    return {"signal": signals, "label": labels, "num_beats": torch.tensor(s_vals, dtype=torch.long)}


def pad_collate_varlen(batch):
    B = len(batch)
    labels = torch.tensor([b["label"] for b in batch], dtype=torch.long)
    S_vals = [b["num_beats"] for b in batch]
    if not all(S_vals[0] == s for s in S_vals):
        raise ValueError("Bucketed sampler should ensure same #beats (S) per batch.")

    S = S_vals[0]
    C = batch[0]["beats"][0].shape[1]
    # T_max across all beats of all items
    T_max = max(beat.shape[0] for item in batch for beat in item["beats"])

    signal = torch.zeros((B, S, T_max, C), dtype=torch.float32)
    mask   = torch.zeros((B, S, T_max),   dtype=torch.float32)

    for bi, item in enumerate(batch):
        for si, beat in enumerate(item["beats"]):
            T = beat.shape[0]
            signal[bi, si, :T, :] = beat
            mask[bi, si, :T] = 1.0

    return {"signal": signal, "mask": mask, "label": labels,
            "num_beats": torch.tensor(S_vals, dtype=torch.long)}


In [ ]:
from torch.utils.data import Subset, DataLoader

# # instantiate dataset and sampler
ds = ECGSegmentDatasetVarLen(data_list)

# -------------------------
# Split indices
# -------------------------
all_indices = list(range(len(ds)))
all_labels = [data_list[i][1] for i in all_indices]  # label_idx for each sample

# First split: train+val vs test (stratified)
trainval_indices, test_indices = train_test_split(
    all_indices,
    test_size=0.1,
    random_state=10,
    stratify=all_labels 
)

# Extract labels for the trainval subset
trainval_labels = [all_labels[i] for i in trainval_indices]

# Second split: train vs val (stratified)
train_indices, val_indices = train_test_split(
    trainval_indices,
    test_size=0.1,
    random_state=10,
    stratify=trainval_labels 
)

# -------------------------
# Create Subsets
# -------------------------
train_ds = Subset(ds, train_indices)
val_ds   = Subset(ds, val_indices)
test_ds  = Subset(ds, test_indices)

# -------------------------
# Make loader helper
# -------------------------
def make_loader(subset_ds, batch_size=16, drop_last=True, shuffle=True):
    subset_indices = subset_ds.indices
    lengths = [ds.num_beats[i] for i in subset_indices]
    sampler = LengthBucketBatchSampler(lengths, batch_size, bin_size=None,
                                       shuffle=shuffle, drop_last=drop_last)
    loader = DataLoader(subset_ds, batch_sampler=sampler,
                        collate_fn=pad_collate_varlen,
                        num_workers=0, pin_memory=True)
    return loader

In [ ]:
# Verifying stratifying
train_loader = make_loader(train_ds, batch_size=BATCH_SIZE)
val_loader   = make_loader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = make_loader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

def get_label_distribution(indices):
    labels = [data_list[i][1] for i in indices]
    return Counter(labels)

print("Train distribution:", get_label_distribution(train_indices))
print("Val distribution:", get_label_distribution(val_indices))
print("Test distribution:", get_label_distribution(test_indices))

Train distribution: Counter({0: 11218, 9: 2190, 1: 2081, 6: 1093, 4: 408, 7: 282, 3: 215, 2: 211, 10: 45, 5: 29, 8: 21})
Val distribution: Counter({0: 1246, 9: 244, 1: 231, 6: 122, 4: 46, 7: 31, 3: 24, 2: 23, 10: 5, 5: 3, 8: 2})
Test distribution: Counter({0: 1385, 9: 271, 1: 257, 6: 135, 4: 50, 7: 35, 2: 26, 3: 26, 10: 6, 8: 3, 5: 3})


In [ ]:
# -------------------------
# Attention / helper layers
# -------------------------
class ChannelAttention(nn.Module):
    """
    Expects x shape = (batch, channels, seq_len)
    """
    def __init__(self, channels, ratio=8):
        super().__init__()
        mid = max(1, channels // ratio)
        self.mlp = nn.Sequential(
            nn.Linear(channels, mid, bias=True),
            nn.ReLU(),
            nn.Linear(mid, channels, bias=True)
        )

    def forward(self, x):
        # x: (B, C, L)
        avg_pool = torch.mean(x, dim=2)           # (B, C)
        max_pool, _ = torch.max(x, dim=2)         # (B, C)
        avg_out = self.mlp(avg_pool)              # (B, C)
        max_out = self.mlp(max_pool)              # (B, C)
        att = torch.sigmoid(avg_out + max_out)    # (B, C)
        att = att.unsqueeze(2)                    # (B, C, 1)
        return x * att                             # broadcast multiply -> (B, C, L)

class SegmentAttention(nn.Module):
    def __init__(self, input_dim, units):
        super().__init__()
        self.linear = nn.Linear(input_dim, units, bias=True)
        self.u = nn.Parameter(torch.randn(units))

    def forward(self, inputs, mask=None):
        """
        inputs: (B, T, D)
        mask:   (B, T) with 1=valid, 0=pad (or None)
        """
        v = torch.tanh(self.linear(inputs))             # (B, T, units)
        vu = torch.matmul(v, self.u)                    # (B, T)
        if mask is not None:
            # set -inf (large negative) where mask==0 so softmax->0
            vu = vu.masked_fill(mask == 0, -1e9)
        alphas = F.softmax(vu, dim=1)                   # (B, T)
        alphas = torch.nan_to_num(alphas, nan=0.0)
        output = torch.sum(inputs * alphas.unsqueeze(-1), dim=1)  # (B, D)
        return output, alphas

class TimeDistributedSegmentAttention(nn.Module):
    def __init__(self, input_dim, units):
        super().__init__()
        self.segment_attention = SegmentAttention(input_dim, units)

    def forward(self, inputs, mask=None):
        """
        inputs: (B, S, T, D)
        mask:   (B, S, T) or None
        """
        B, S, T, D = inputs.shape
        flat = inputs.view(B * S, T, D)                         # (B*S, T, D)
        if mask is not None:
            mask_flat = mask.view(B * S, T)                     # (B*S, T)
        else:
            mask_flat = None
        outputs, alphas = self.segment_attention(flat, mask_flat)  # (B*S, D), (B*S, T)
        outputs = outputs.view(B, S, D)                         # (B, S, D)
        alphas  = alphas.view(B, S, T)                          # (B, S, T)
        return outputs, alphas

class HANWithAttention(nn.Module):
    def __init__(self, num_classes=11):
        super().__init__()
        self.conv1d = nn.Conv1d(in_channels=12, out_channels=128, kernel_size=25, padding=12)
        self.bn1 = nn.BatchNorm1d(128)
        self.channel_attention = ChannelAttention(128, ratio=8)
        self.pool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.lstm_segment = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.time_distributed_attention = TimeDistributedSegmentAttention(input_dim=256, units=256)
        self.lstm_sequence = nn.LSTM(input_size=256, hidden_size=512, batch_first=True)
        self.final_attention = SegmentAttention(input_dim=512, units=512)
        self.fc = nn.Linear(512, 2048)
        self.dropout = nn.Dropout(0.4)
        self.classifier = nn.Linear(2048, num_classes)

    def forward(self, x, mask=None):
        logits, _, _ = self.forward_with_attention(x, mask)
        return logits

    def forward_with_attention(self, x, mask=None):
        """
        x:    (B, S, T, C)
        mask: (B, S, T) with 1=valid, 0=pad (or None)
        """
        B, S, T, C = x.shape
        x = x.view(B * S, T, C).permute(0, 2, 1)    # (B*S, C, T)

        conv = self.conv1d(x)                       # (B*S, 128, T)
        conv = self.bn1(conv)
        conv = F.relu(conv)
        att  = self.channel_attention(conv)         # (B*S, 128, T)
        pooled = self.pool(att)                     # (B*S, 128, T2)
        pooled = pooled.permute(0, 2, 1)            # (B*S, T2, 128)

        # Downsample mask to T2 with the same pooling parameters
        if mask is not None:
            m = mask.view(B * S, 1, T)              # (B*S, 1, T)
            m2 = F.max_pool1d(m, kernel_size=3, stride=2, padding=1)  # (B*S, 1, T2)
            m2 = (m2 > 0.0).float().squeeze(1)      # (B*S, T2)
        else:
            m2 = None

        seg_lstm_out, _ = self.lstm_segment(pooled) # (B*S, T2, 256)
        seg_lstm_out = seg_lstm_out.view(B, S, seg_lstm_out.shape[1], seg_lstm_out.shape[2])  # (B, S, T2, 256)
        if m2 is not None:
            m2 = m2.view(B, S, -1)                  # (B, S, T2)

        segment_outputs, segment_alphas = self.time_distributed_attention(seg_lstm_out, mask=m2)  # (B, S, 256), (B, S, T2)
        seq_lstm_out, _ = self.lstm_sequence(segment_outputs)                                     # (B, S, 512)
        final_output, final_alphas = self.final_attention(seq_lstm_out)                           # (B, 512), (B, S)

        x = F.relu(self.fc(final_output))
        x = self.dropout(x)
        logits = self.classifier(x)                 # (B, num_classes)
        return logits, final_alphas, segment_alphas


In [ ]:
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HANWithAttention(num_classes=11).to(device)

summary(model,
        input_size=(2, 10, 300, 12),    # this wouldn't be the actual size for r-r interval splitting
        col_names=("input_size", "output_size", "num_params", "trainable"),
        depth=4,
        device=device.type)            # "cpu" or "cuda"


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
HANWithAttention                              [2, 10, 300, 12]          [2, 11]                   --                        True
├─Conv1d: 1-1                                 [20, 12, 300]             [20, 128, 300]            38,528                    True
├─BatchNorm1d: 1-2                            [20, 128, 300]            [20, 128, 300]            256                       True
├─ChannelAttention: 1-3                       [20, 128, 300]            [20, 128, 300]            --                        True
│    └─Sequential: 2-1                        [20, 128]                 [20, 128]                 --                        True
│    │    └─Linear: 3-1                       [20, 128]                 [20, 16]                  2,064                     True
│    │    └─ReLU: 3-2                         [20, 16]                  [20, 16]            

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

from tqdm import tqdm

def evaluate_metrics(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels, all_preds, all_probs = [], [], []

    with torch.no_grad():
        for batch in loader:
            inputs = batch["signal"].to(device)
            labels = batch["label"].to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            probs = torch.softmax(outputs, dim=1).cpu().numpy()
            preds = outputs.argmax(dim=1).cpu().numpy()
            labels = labels.cpu().numpy()

            all_labels.extend(labels)
            all_preds.extend(preds)
            all_probs.extend(probs)

    avg_loss = running_loss / len(all_labels)
    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
    recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
    f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)

    try:
      y_true = np.eye(num_classes)[all_labels]
      y_score = np.array(all_probs)

      auc_list = []
      for i in range(num_classes):
          if np.any(y_true[:, i]):  # class i exists
              auc_list.append(roc_auc_score(y_true[:, i], y_score[:, i]))
      if auc_list:
          auc = np.mean(auc_list)
      else:
          auc = float("nan")
    except ValueError:
        auc = float("nan")

    return avg_loss, acc, precision, recall, auc, f1

def train(model, train_loader, val_loader, optimizer, criterion, device,
          epochs, scheduler=None):
    model.to(device)
    history = {
        "train_loss": [], "train_acc": [], "train_f1": [], "train_precision": [], "train_recall": [], "train_auc": [],
        "val_loss": [],   "val_acc": [],   "val_f1": [],   "val_precision": [],   "val_recall": [], "val_auc": []
    }

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        all_preds = []
        all_probs = []
        all_labels = []
        steps = 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
        for batch in loop:
            inputs = batch["signal"].to(device)     # (B,S,T_max,C)
            mask   = batch["mask"].to(device)       # (B,S,T_max)
            labels = batch["label"].to(device)  
            optimizer.zero_grad(set_to_none=True)
            outputs = model(inputs, mask=mask)

            loss = criterion(outputs, labels)
            loss.backward()

            optimizer.step()

            batch_size = labels.size(0)
            running_loss += loss.item() * batch_size
            preds = outputs.argmax(dim=1)
            probs = torch.softmax(outputs, dim=1).detach().cpu().numpy()
            correct += (preds == labels).sum().item()
            total += batch_size

            all_preds.extend(preds.cpu().numpy().tolist())
            all_probs.extend(probs.tolist())
            all_labels.extend(labels.cpu().numpy().tolist())

            steps += 1
            loop.set_postfix(loss=f"{loss.item():.4f}", acc=f"{(correct/total):.4f}")

        if scheduler is not None:
            try:
                scheduler.step()
            except Exception:
                pass

        if total == 0:
            epoch_loss = float("nan")
            epoch_acc = float("nan")
            epoch_precision = epoch_recall = epoch_f1 = float("nan")
        else:
            epoch_loss = running_loss / total
            epoch_acc = correct / total
            epoch_precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
            epoch_recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
            epoch_f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)
            try:
              y_true = np.eye(num_classes)[all_labels]
              y_score = np.array(all_probs)

              auc_list = []
              for i in range(num_classes):
                  if np.any(y_true[:, i]):
                      auc_list.append(roc_auc_score(y_true[:, i], y_score[:, i]))
              if auc_list:
                  auc = np.mean(auc_list)
              else:
                  auc = float("nan")
            except ValueError:
                auc = float("nan")

        # append train history
        history["train_loss"].append(epoch_loss)
        history["train_auc"].append(auc)
        history["train_acc"].append(epoch_acc)
        history["train_precision"].append(epoch_precision)
        history["train_recall"].append(epoch_recall)
        history["train_f1"].append(epoch_f1)

        print(f"Epoch {epoch}/{epochs}  train_loss: {epoch_loss:.4f}  train_acc: {epoch_acc:.4f}  train_f1: {epoch_f1:.4f}")

        # Validation
        if val_loader is not None:
            vloss, vacc, vprecision, vrecall, vauc, vf1 = evaluate_metrics(model, val_loader, criterion, device)
            history["val_loss"].append(vloss)
            history["val_acc"].append(vacc)
            history["val_auc"].append(vauc)
            history["val_f1"].append(vf1)
            history["val_precision"].append(vprecision)
            history["val_recall"].append(vrecall)


            # log per-epoch metrics to mlflow if available
            if 'mlflow' in globals():
                try:
                    mlflow.log_metric("train_loss", epoch_loss, step=epoch)
                    mlflow.log_metric("train_f1", epoch_f1, step=epoch)
                    mlflow.log_metric("train_precision", epoch_precision, step=epoch)
                    mlflow.log_metric("train_recall", epoch_recall, step=epoch)
                    mlflow.log_metric("train_acc", epoch_acc, step=epoch)
                    mlflow.log_metric("train_auc", auc, step=epoch)

                    mlflow.log_metric("val_loss", vloss, step=epoch)
                    mlflow.log_metric("val_f1", vf1, step=epoch)
                    mlflow.log_metric("val_precision", vprecision, step=epoch)
                    mlflow.log_metric("val_recall", vrecall, step=epoch)
                    mlflow.log_metric("val_acc", vacc, step=epoch)
                    mlflow.log_metric("val_auc", vauc, step=epoch)
                except Exception:
                    pass

            tqdm.write(
                f"Epoch {epoch}  train_loss {epoch_loss:.4f} | val_loss {vloss:.4f} | "
                f"val_f1 {vf1:.4f} | val_precision {vprecision:.4f} | val_recall {vrecall:.4f} | val_auc {vauc:.4f}"
            )
        else:
            # keep consistent history lengths
            history["val_loss"].append(None)
            history["val_auc"].append(None)
            history["val_acc"].append(None)
            history["val_f1"].append(None)
            history["val_precision"].append(None)
            history["val_recall"].append(None)

    return history

# -------------


In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from tqdm import tqdm
import random

def run_experiments(model_fn,
                    optimizer_fn, criterion, device, epochs, num_classes,
                    seeds=10):

  results = {"acc": [], "precision": [], "recall": [], "auc": [], "loss": []}
  print("run exp")
  for seed in range(4, 10 ):
      print(f"\n=== Seed {seed} ===")
      set_seed(seed)
      ds = ECGSegmentDatasetVarLen(data_list)

      all_indices = list(range(len(ds)))
      all_labels = [data_list[i][1] for i in all_indices]  # label_idx for each sample

      # First split: train+val vs test (stratified)
      trainval_indices, test_indices = train_test_split(
          all_indices,
          test_size=0.1,
          random_state=seed,
          stratify=all_labels  # ← stratify by class labels
      )

      # Extract labels for the trainval subset
      trainval_labels = [all_labels[i] for i in trainval_indices]

      # Second split: train vs val (stratified)
      train_indices, val_indices = train_test_split(
          trainval_indices,
          test_size=0.1,
          random_state=seed,
          stratify=trainval_labels  # ← stratify by class labels
      )

      # -------------------------
      # Create Subsets
      # -------------------------
      train_ds = Subset(ds, train_indices)
      val_ds   = Subset(ds, val_indices)
      test_ds  = Subset(ds, test_indices)

      train_loader = make_loader(train_ds, batch_size=BATCH_SIZE)
      val_loader   = make_loader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
      test_loader  = make_loader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

      print("Train distribution:", get_label_distribution(train_indices))
      print("Val distribution:", get_label_distribution(val_indices))
      print("Test distribution:", get_label_distribution(test_indices))

      model = model_fn().to(device)
      optimizer = optimizer_fn(model)
      scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

      # Train
      with mlflow.start_run(nested=True, run_name=f"HAN-SPH-SEED-{seed}"):
        _ = train(model, train_loader, val_loader, optimizer, criterion, device, epochs, scheduler)

        # Evaluate on test set
        loss, acc, precision, recall, auc, f1 = evaluate_metrics(model, test_loader, criterion, device)

        print(f"Test (seed {seed}) — loss: {loss:.4f}, acc: {acc:.4f}, "
            f"precision: {precision:.4f}, recall: {recall:.4f}, auc: {auc:.4f}, f1: {f1:.4f}")

      results["loss"].append(loss)
      results["acc"].append(acc)
      results["precision"].append(precision)
      results["recall"].append(recall)
      results["auc"].append(auc)

  # Aggregate results
  print("\n=== Final Results (across seeds) ===")
  for k, v in results.items():
      arr = np.array(v, dtype=np.float32)
      print(f"{k}: mean={arr.mean():.4f}, std={arr.std():.4f}")

  return results


In [ ]:
def model_fn():
  return HANWithAttention(num_classes=num_classes)

def optimizer_fn(model):
  return torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-3)


In [ ]:
criterion = nn.CrossEntropyLoss()
run_experiments(model_fn, optimizer_fn, criterion, DEVICE, 30, 11, 10)


In [ ]:
summary(model,
        input_size=(2, 10, 300, 12),    # (batch, segments, timesteps, channels)
        col_names=("input_size", "output_size", "num_params", "trainable"),
        depth=4,
        device=device.type)

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Trainable
HANWithAttention                              [2, 10, 300, 12]          [2, 11]                   --                        True
├─Conv1d: 1-1                                 [20, 12, 300]             [20, 128, 300]            38,528                    True
├─ChannelAttention: 1-2                       [20, 128, 300]            [20, 128, 300]            --                        True
│    └─Sequential: 2-1                        [20, 128]                 [20, 128]                 --                        True
│    │    └─Linear: 3-1                       [20, 128]                 [20, 16]                  2,064                     True
│    │    └─ReLU: 3-2                         [20, 16]                  [20, 16]                  --                        --
│    │    └─Linear: 3-3                       [20, 16]                  [20, 128]             

# Resource use

In [ ]:
!pip install calflops -q

In [ ]:
import calflops

In [ ]:
model = HANWithAttention(num_classes=11)

flops, macs, params = calflops.calculate_flops(model=model,
                                      input_shape=(1, 10, 300, 12),
                                      output_as_string=True,
                                      output_precision=4)
print("Model FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))


------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  3.42 M  
fwd MACs:                                                               318.204 MMACs
fwd FLOPs:                                                              1.8522 GFLOPS
fwd+bwd MACs:                                                           954.612 MMACs
fwd+bwd FLOPs:                                                          5.5565 GFLOPS

-------------------------------- Detailed Calculated FLOPs Results --------------------------------
Each modu

In [ ]:
def analyze_model_complexity(model, input_size=(1, 10, 300, 12), device='cuda'):
    """
    Analyze model complexity: parameters, FLOPs, memory

    Args:
        model: Your HANWithAttention model
        input_size: (batch, segments, timesteps, channels)
        device: 'cuda' or 'cpu'
    """
    print("="*80)
    print("MODEL COMPLEXITY ANALYSIS")
    print("="*80)

    model = model.to(device)
    model.eval()

    # 1. Parameter count
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params

    print(f"\n📊 Parameter Count:")
    print(f"   Total Parameters:       {total_params:,}")
    print(f"   Trainable Parameters:   {trainable_params:,}")
    print(f"   Non-trainable Parameters: {non_trainable_params:,}")
    print(f"   Model Size (MB):        {total_params * 4 / (1024**2):.2f}")  # 4 bytes per float32

    # 2. FLOPs calculation
    dummy_input = torch.randn(input_size).to(device)

    try:
        macs, params = profile(model, inputs=(dummy_input,), verbose=False)
        macs, params = clever_format([macs, params], "%.3f")
        print(f"\n🔢 FLOPs (Floating Point Operations):")
        print(f"   MACs (Multiply-Accumulate): {macs}")
        print(f"   Parameters (thop):          {params}")
    except Exception as e:
        print(f"\n⚠️  FLOPs calculation failed: {e}")

    # 3. Layer-wise parameter breakdown
    print(f"\n📋 Layer-wise Parameter Breakdown:")
    print(f"{'Layer Name':<40} {'Parameters':>15} {'% of Total':>12}")
    print("-"*70)

    for name, param in model.named_parameters():
        if param.requires_grad:
            param_count = param.numel()
            percentage = 100 * param_count / trainable_params
            print(f"{name:<40} {param_count:>15,} {percentage:>11.2f}%")

    # 4. Detailed model summary
    print(f"\n📝 Detailed Model Architecture:")
    summary(model,
            input_size=input_size,
            col_names=["input_size", "output_size", "num_params", "mult_adds"],
            depth=4,
            device=device,
            verbose=0)

    return {
        'total_params': total_params,
        'trainable_params': trainable_params,
        'model_size_mb': total_params * 4 / (1024**2)
    }

analyze_model_complexity(model)

MODEL COMPLEXITY ANALYSIS

📊 Parameter Count:
   Total Parameters:       3,417,371
   Trainable Parameters:   3,417,371
   Non-trainable Parameters: 0
   Model Size (MB):        13.04

🔢 FLOPs (Floating Point Operations):
   MACs (Multiply-Accumulate): 829.057M
   Parameters (thop):          3.417M

📋 Layer-wise Parameter Breakdown:
Layer Name                                    Parameters   % of Total
----------------------------------------------------------------------
conv1d.weight                                     38,400        1.12%
conv1d.bias                                          128        0.00%
channel_attention.mlp.0.weight                     2,048        0.06%
channel_attention.mlp.0.bias                          16        0.00%
channel_attention.mlp.2.weight                     2,048        0.06%
channel_attention.mlp.2.bias                         128        0.00%
lstm_segment.weight_ih_l0                        131,072        3.84%
lstm_segment.weight_hh_l0         

{'total_params': 3417371,
 'trainable_params': 3417371,
 'model_size_mb': 13.036235809326172}

In [ ]:
# From: https://discuss.pytorch.org/t/finding-model-size/130275

param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))


model size: 13.036MB


In [ ]:
import time
import numpy as np
import torch

# =============================================================================
# Benchmark Environment Setup
# =============================================================================

def setup_benchmark_env():
    """
    Freeze backend behavior for reproducible benchmarking.
    """
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False

def time_forward(model, inputs, device='cuda'):
    """
    Time a single forward pass with proper synchronization.
    Returns elapsed time in seconds.
    """
    if device == 'cuda':
        torch.cuda.synchronize()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        _ = model(inputs)
        end.record()

        torch.cuda.synchronize()
        return start.elapsed_time(end) / 1000.0  # seconds
    else:
        t0 = time.time()
        _ = model(inputs)
        return time.time() - t0


# =============================================================================
# Batch-1 Latency Benchmark
# =============================================================================

def benchmark_latency(
    model,
    input_shape,
    device='cuda',
    runs=100,
    warmup=20
):
    """
    Measure true per-sample inference latency (batch size = 1).
    """
    model = model.to(device).eval()

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(input_shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(input_shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "mean_ms": times.mean() * 1000,
        "std_ms": times.std() * 1000,
        "median_ms": np.median(times) * 1000,
        "min_ms": times.min() * 1000,
        "max_ms": times.max() * 1000,
    }


# =============================================================================
# Batch-N Throughput Benchmark
# =============================================================================

def benchmark_throughput(
    model,
    input_shape,
    batch_size,
    device='cuda',
    runs=100,
    warmup=20
):
    """
    Measure throughput under batched inference.
    """
    model = model.to(device).eval()
    shape = (batch_size,) + input_shape[1:]

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "batch_time_ms": times.mean() * 1000,
        "per_sample_ms": (times.mean() / batch_size) * 1000,
        "samples_per_sec": batch_size / times.mean()
    }

def run_inference_benchmark(
    model,
    input_shape=(1, 10, 300, 12),
    batch_size=16,
    device='cuda'
):
    setup_benchmark_env()

    print("=" * 80)
    print("INFERENCE BENCHMARK")
    print("=" * 80)
    print(f"Device: {device}")
    print(f"Input shape: {input_shape}")
    print(f"Throughput batch size: {batch_size}")

    latency = benchmark_latency(
        model=model,
        input_shape=input_shape,
        device=device
    )

    throughput = benchmark_throughput(
        model=model,
        input_shape=input_shape,
        batch_size=batch_size,
        device=device
    )

    print("\nBatch-1 Latency (ms):")
    for k, v in latency.items():
        print(f"  {k:>10}: {v:.3f}")

    print("\nBatch-N Throughput:")
    for k, v in throughput.items():
        if "ms" in k:
            print(f"  {k:>16}: {v:.3f}")
        else:
            print(f"  {k:>16}: {v:.2f}")

    return {
        "latency": latency,
        "throughput": throughput
    }

results = run_inference_benchmark(
    model,
    input_shape=(1, 10, 300, 12),
    batch_size=16,
    device='cuda'
)


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


INFERENCE BENCHMARK
Device: cuda
Input shape: (1, 10, 300, 12)
Throughput batch size: 16

Batch-1 Latency (ms):
     mean_ms: 3.188
      std_ms: 0.174
   median_ms: 3.162
      min_ms: 3.072
      max_ms: 4.449

Batch-N Throughput:
     batch_time_ms: 6.826
     per_sample_ms: 0.427
   samples_per_sec: 2343.96


In [ ]:

results = run_inference_benchmark(
    model,
    input_shape=(1, 10, 300, 12),
    batch_size=16,
    device='cpu'
)

INFERENCE BENCHMARK
Device: cpu
Input shape: (1, 10, 300, 12)
Throughput batch size: 16

Batch-1 Latency (ms):
     mean_ms: 10.316
      std_ms: 0.344
   median_ms: 10.247
      min_ms: 9.640
      max_ms: 11.711

Batch-N Throughput:
     batch_time_ms: 128.110
     per_sample_ms: 8.007
   samples_per_sec: 124.89
